# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Li  ->  J. Li  |  ['J. Li']
T. Cantat-Gaudin  ->  T. Cantat-Gaudin  |  ['T. Cantat-Gaudin']
S. Ghosh  ->  S. Ghosh  |  ['S. Ghosh']
Arxiv has 92 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2311.02607


extracting tarball to tmp_2311.02607...

 done.
Retrieving document from  https://arxiv.org/e-print/2311.03009


extracting tarball to tmp_2311.03009...

 done.


T. Cantat-Gaudin  ->  T. Cantat-Gaudin  |  ['T. Cantat-Gaudin']


Found 115 bibliographic references in tmp_2311.03009/sample631.bbl.
syntax error in line 494: '=' expected
Retrieving document from  https://arxiv.org/e-print/2311.03023


extracting tarball to tmp_2311.03023...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2311.03009-b31b1b.svg)](https://arxiv.org/abs/arXiv:2311.03009) | **Parameter Estimation for Open Clusters using an Artificial Neural  Network with a QuadTree-based Feature Extractor**  |
|| L. Cavallo, et al. -- incl., <mark>T. Cantat-Gaudin</mark> |
|*Appeared on*| *2023-11-07*|
|*Comments*| *24 pages, 15 figures, Accepted in The Astronomical Journal. Temporally, data produced in this work are available at this https URL*|
|**Abstract**| With the unprecedented increase of known star clusters, quick and modern tools are needed for their analysis. In this work, we develop an artificial neural network trained on synthetic clusters to estimate the age, metallicity, extinction, and distance of $Gaia$ open clusters. We implement a novel technique to extract features from the colour-magnitude diagram of clusters by means of the QuadTree tool and we adopt a multi-band approach. We obtain reliable parameters for $\sim 5400$ clusters. We demonstrate the effectiveness of our methodology in accurately determining crucial parameters of $Gaia$ open clusters by performing a comprehensive scientific validation. In particular, with our analysis we have been able to reproduce the Galactic metallicity gradient as it is observed by high-resolution spectroscopic surveys. This demonstrates that our method reliably extracts information on metallicity from colour-magnitude diagrams (CMDs) of stellar clusters. For the sample of clusters studied, we find an intriguing systematic older age compared to previous analyses present in the literature. This work introduces a novel approach to feature extraction using a QuadTree algorithm, effectively tracing sequences in CMDs despite photometric errors and outliers. The adoption of ANNs, rather than Convolutional Neural Networks, maintains the full positional information and improves performance, while also demonstrating the potential for deriving clusters' parameters from simultaneous analysis of multiple photometric bands, beneficial for upcoming telescopes like the Vera Rubin Observatory. The implementation of ANN tools with robust isochrone fit techniques could provide further improvements in the quest for open clusters' parameters. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2311.02607-b31b1b.svg)](https://arxiv.org/abs/arXiv:2311.02607) | **AI Techniques for Uncovering Resolved Planetary Nebula Candidates from  Wide-field VPHAS+ Survey Data**  |
|| R. Sun, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2023-11-07*|
|*Comments*| **|
|**Abstract**| AI and deep learning techniques are beginning to play an increasing role in astronomy as a necessary tool to deal with the data avalanche. Here we describe an application for finding resolved Planetary Nebulae (PNe) in crowded, wide-field, narrow-band H-alpha survey imagery in the Galactic plane. PNe are important to study late stage of stellar evolution of low to intermediate-mass stars. However, the confirmed ~3800 Galactic PNe fall far short of the numbers expected. Traditional visual searching for resolved PNe is time-consuming due to the large data size and areal coverage of modern astronomical surveys, especially those taken in narrow-band filters highlighting emission nebulae. To test and facilitate more objective, reproducible, efficient and reliable trawls for PNe candidates we have developed a new, deep learning algorithm. In this paper, we applied the algorithm to several H-alpha digital surveys (e.g. IPHAS and VPHAS+). The training and validation dataset was built with true PNe from the HASH database. After transfer learning, it was then applied to the VPHAS+ survey. We examined 979 out of 2284 survey fields with each survey field covering 1 * 1 deg^2. With a sample of 454 PNe from the IPHAS as our validation set, our algorithm correctly identified 444 of these objects (97.8%), with only 16 explicable 'false' positives. Our model returned ~20,000 detections, including 2637 known PNe and many other kinds of catalogued non-PNe such as HII regions. A total of 815 new high-quality PNe candidates were found, 31 of which were selected as top-quality targets for subsequent optical spectroscopic follow-up. Representative preliminary confirmatory spectroscopy results are presented here to demonstrate the effectiveness of our techniques with full details to be given in paper-II. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2311.03023-b31b1b.svg)](https://arxiv.org/abs/arXiv:2311.03023) | **Transient RFI environment of LOFAR-LBA at 72-75 MHz: Impact on  ultra-widefield AARTFAAC Cosmic Explorer observations of the redshifted 21-cm  signal**  |
|| B. Gehlot, et al. -- incl., <mark>S. Ghosh</mark> |
|*Appeared on*| *2023-11-07*|
|*Comments*| *13 pages, 9 figures, and 3 tables; accepted for publication in Astronomy and Astrophysics (A&A)*|
|**Abstract**| Measurement of the redshifted 21-cm signal of neutral hydrogen from the Cosmic Dawn (CD) and Epoch of Reionisation (EoR) promises to unveil a wealth of information about the astrophysical processes during the first billion years of evolution of the universe. The AARTFAAC Cosmic Explorer (ACE) utilises the AARTFAAC wide-field imager of LOFAR to measure the power spectrum of the intensity fluctuations of the redshifted 21-cm signal from the CD at z~18. The RFI from various sources contaminates the observed data and it is crucial to exclude the RFI-affected data in the analysis for reliable detection. In this work, we investigate the impact of non-ground-based transient RFI using cross-power spectra and cross-coherence metrics to assess the correlation of RFI over time and investigate the level of impact of transient RFI on the ACE 21-cm power spectrum estimation. We detected moving sky-based transient RFI sources that cross the field of view within a few minutes and appear to be mainly from aeroplane communication beacons at the location of the LOFAR core in the 72-75 MHz band, by inspecting filtered images. This transient RFI is mostly uncorrelated over time and is only expected to dominate over the thermal noise for an extremely deep integration time of 3000 hours or more with a hypothetical instrument that is sky temperature dominated at 75 MHz. We find no visible correlation over different k-modes in Fourier space in the presence of noise for realistic thermal noise scenarios. We conclude that the sky-based transient RFI from aeroplanes, satellites and meteorites at present does not pose a significant concern for the ACE analyses at the current level of sensitivity and after integrating over the available 500 hours of observed data. However, it is crucial to mitigate or filter such transient RFI for more sensitive experiments aiming for significantly deeper integration. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2311.03009.md
    + _build/html/tmp_2311.03009/./age_distribution.png
    + _build/html/tmp_2311.03009/./comparison_literature.png
    + _build/html/tmp_2311.03009/./examples.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\}{cg}$
$\newcommand{\}{ep}$
$\newcommand{\}{lm}$
$\newcommand{\}{lc}$
$\newcommand{\}{ls}$
$\newcommand{\}{mp}$
$\newcommand{\}{lg}$
$\newcommand{\}{sl}$
$\newcommand{\gaia}{\textit{Gaia}}$
$\newcommand{\gbp}{G_{\rm BP}}$
$\newcommand{\grp}{G_{\rm RP}}$
$\newcommand{\mudist}{\mu_{\rm dist}}$</div>



<div id="title">

# Parameter Estimation for Open Clusters using an Artificial Neural Network with a QuadTree-based Feature Extractor

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2311.03009-b31b1b.svg)](https://arxiv.org/abs/2311.03009)<mark>Appeared on: 2023-11-07</mark> -  _24 pages, 15 figures, Accepted in The Astronomical Journal. Temporally, data produced in this work are available at this https URL_

</div>
<div id="authors">

L. Cavallo, et al. -- incl., <mark>T. Cantat-Gaudin</mark>

</div>
<div id="abstract">

**Abstract:** With the unprecedented increase of known star clusters, quick and modern tools are needed for their analysis. In this work, we develop an artificial neural network trained on synthetic clusters to estimate the age, metallicity, extinction, and distance of $\gaia$ open clusters. We implement a novel technique to extract features from the colour-magnitude diagram of clusters by means of the QuadTree tool and we adopt a multi-band approach. We obtain reliable parameters for $\sim 5400$ clusters $\footnote{The catalogue of clusters analysed in this work is available at \url{https://phisicslollo0.github.io/cavallo23.html}. See Appendix \ref{sec:appendix} for more details.}$ . We demonstrate the effectiveness of our methodology in accurately determining crucial parameters of $\gaia$ open clusters by performing a comprehensive scientific validation. In particular, with our analysis we have been able to reproduce the Galactic metallicity gradient as it is observed by high-resolution spectroscopic surveys. This demonstrates that our method reliably extracts information on metallicity from colour-magnitude diagrams (CMDs) of stellar clusters. For the sample of clusters studied, we find an intriguing systematic older age compared to previous analyses present in the literature. This work introduces a novel approach to feature extraction using a QuadTree algorithm, effectively tracing sequences in CMDs despite photometric errors and outliers. The adoption of ANNs, rather than Convolutional Neural Networks, maintains the full positional information and improves performance, while also demonstrating the potential for deriving clusters' parameters from simultaneous analysis of multiple photometric bands, beneficial for upcoming telescopes like the Vera Rubin Observatory. The implementation of ANN tools with robust isochrone fit techniques could provide further improvements in the quest for open clusters' parameters.

</div>

<div id="div_fig1">

<img src="tmp_2311.03009/./age_distribution.png" alt="Fig9" width="100%"/>

**Figure 9. -** Age distributions of $\gaia$ open clusters as predicted by \citetalias{CG20}, \citetalias{D21}, \citetalias{H23}, and in this work. Left panel: differential age distribution. Right panel: cumulative age distribution. (*fig:age_distribution*)

</div>
<div id="div_fig2">

<img src="tmp_2311.03009/./comparison_literature.png" alt="Fig8" width="100%"/>

**Figure 8. -** Comparison between parameters derived in this work with the ones present in literature. In the top left corners is reported the value of the root main square error. Top row: comparison of age, extinction, and distance for clusters that are in common with \citetalias{CG20}. Middle row: comparison of age, [Fe/H], extinction, and distance for clusters in common with \citetalias{D21}. Bottom row: comparison of age, extinction, and distance for clusters contained in \citetalias{H23}. (*fig:comparison*)

</div>
<div id="div_fig3">

<img src="tmp_2311.03009/./examples.png" alt="Fig10" width="100%"/>

**Figure 10. -** Predicted isochrones for three open clusters UBC 276  (left panel), UBC 562 (middle panel), and Teutsch 30 (right panel) from \citetalias{CG20}(dot-dashed green line), \citetalias{D21}(dashed orange line), \citetalias{H23}(solid purple line), and this work (solid red line). In the top right, we annotate the ages that correspond to the plotted isochrones (coloured accordingly). With blue and full dots we plot the members of the clusters retrieved by \citetalias{H23}, with empty black dots are plotted the ones of \citetalias{D21}. (*fig:examples*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2311.03009"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

317  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

7  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
